In [4]:
import os
import sys
parent = os.path.realpath(os.path.join(os.path.abspath(''), os.pardir))
sys.path.append(parent)
from databases.sql_connect import create_connection

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

from copy import deepcopy
import plotly.graph_objects as go
import plotly.express as pex
import holoviews as hv
from holoviews import dim
from bokeh.io import export_png


import warnings
from copy import deepcopy
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option("mode.chained_assignment", None) 

from floweaver import *
from ipysankeywidget import SankeyWidget

conn = None
sns.set_theme()
sns.set(context="paper", style="darkgrid", font="Times New Roman")
cm = 1/2.54
DB_CORR = os.path.join(parent,"databases", "TEMPRO_DB230416_Corr.db")
assert(os.path.exists(DB_CORR))
FONTSIZE = 10

cats = {"CED - total": "CED",
        "CExD - metals": "CExD",
        "ADP": "ADP",
        "EI-Minerals- Total": "ADP-EI",
        "GPR-Minerals- Total": "GPSR",
        "CML-climate change (GWP 100a)": "GWP",
        "ReCiPe Midpoint (E) - metal depletion (MDP)": "MDP",
        "ReCiPe Endpoint (E - A) - total (total)": "Total",
        }

full_label = {
    'CED': 'Primary Energy Demand [MJ-eq]',
    'CExD': 'Primary Exergy Demand [MJ-eq]',
    'ADP': 'Abiotic Depletion Potential [kg Sb-eq]',
    "ADP-EI": 'ADP - Economic Importance [kg Sb-eq]',
    'GPSR': 'Geo-Political Suppy Risk [1]',
    'GWP': 'Greenhouse Emissions [kg CO2-eq]',
    'MDP': 'Metal Depletion Potential [kg Fe-eq]',
    'Total': 'ReCiPe-Total Impacts',
}


sns.set_theme(font_scale=1)
sns.set(context="paper", style="darkgrid", font="Times New Roman")
cm = 1/2.54
DB_CORR = os.path.join(parent,"databases", "TEMPRO_DB230515_Corr.db")
assert(os.path.exists(DB_CORR))

In [5]:
def get_lca_contributions(ps_list, category, group=6000):
    conn = create_connection(DB_CORR)
    query = f'SELECT * FROM [{group}LCAResults]'
    lca = pd.read_sql_query(query, conn)
    #ps_list = [p for p in ps_list if p in lca["ProductSystemID"]]
    lca = lca[lca["ProductSystemID"].isin(ps_list)]
    lca = lca[lca["Category"] == category]
    lca.set_index("ProductSystemID", inplace=True)
    #lca.drop(columns=["ID","StandardDev","DataQuality"], inplace=True)
    conn.close()
    return lca

def get_inventory(group):
    conn = create_connection(DB_CORR)
    queryInv = f'SELECT * FROM [{group}Exchanges]'
    inv = pd.read_sql_query(queryInv, conn)
    inv["Contribution"] = np.nan
    conn.close()
    return inv

def get_impacts(ps):
    group = ps- ps%1000
    conn = create_connection(DB_CORR)
    query = f'SELECT * FROM [{group}LCAResults]'
    inv = pd.read_sql_query(query, conn)
    inv= inv[inv["ProductSystemID"]==ps]
    #inv.set_index("Category", inplace =  True)
    inv = inv[["Category","ImpactID","Result"]]
    
    conn.close()
    return inv

def get_ps_info(group= 4000):
    conn = create_connection(DB_CORR)
    table_name = {2000:"2000Parts",
                  3000:"3000Pieces",
                  4000:"4000Devices",
                5000:"5000Systems",
                                6000:"6000DataCenters",
                                7000: "7000Recovery"}
    query = f'SELECT * FROM [{table_name[group]}]'
    df_ps = pd.read_sql_query(query, conn)
    conn.close() 
    df_ps.set_index("ID", inplace=True)
    return df_ps



In [6]:

for PS_ID in [3204, 3208,3209,4401,4433,4440,5101,6001]:
    df_ps_impacts= get_impacts(PS_ID)
    df_7000 = get_ps_info(7000)
    df_flows = df_7000[df_7000["Type"]=="RawMat"]

    ps_list = list(df_flows.index)

    for c in cats.keys():
        df_flows[c] = np.nan
        a = get_lca_contributions(ps_list, c, group=7000)
        for idx in df_flows.index:
            df_flows.at[idx,c] = a.at[idx,"Result"]


    mat_ =[]
    for idx in df_flows.index:
        name = str(idx) + ":" + df_flows.at[idx,"Name"]
        mat_.append(name)
    df_ps_impacts = df_ps_impacts[df_ps_impacts["Category"].isin(mat_)]
    df_ps_impacts["ID"] = np.nan
    for idx in df_ps_impacts.index:
        df_ps_impacts.at[idx,"ID"] = int(df_ps_impacts.at[idx,"Category"][0:4])

    df_ps_impacts.set_index("ID", inplace=True)

    # ####
    # df_ps_impacts["MatName"]= df_ps_impacts["Category"].apply(lambda x: x[5:])


    # conn = create_connection(DB_CORR)
    # query = f'SELECT * FROM [1000LCAResults]'
    # lca = pd.read_sql_query(query, conn)
    # conn.close()
    # ### get material impacts:
    # for cat in cats.keys():

    #     #lca = lca[lca["ProductSystemID"].isin(ps_list)]
    #     lca_cat = lca[lca["Category"] == cat]
    #     lca_cat.set_index("ProductSystemName", inplace=True)
    #     df_ps_impacts[cat]=np.nan
    #     for idx in df_ps_impacts.index:
    #         name = df_ps_impacts.at[idx,"MatName"]
    #         if name in lca_cat.index:
    #             df_ps_impacts.at[idx,cat]= lca_cat.at[name,"Result"]*df_ps_impacts.at[idx,"Result"]
    #         else:
    #             df_ps_impacts.at[idx,cat]=0

    # df_ps_impacts



    # conn = create_connection(DB_CORR)
    # query = f'SELECT * FROM [1000LCAResults]'
    # lca = pd.read_sql_query(query, conn)
    # conn.close()
    # ### get material impacts:
    # for cat in cats.keys()_:

    #     #ps_list = [p for p in ps_list if p in lca["ProductSystemID"]]
    #     lca = lca[lca["ProductSystemID"].isin(ps_list)]
    #     lca = lca[lca["Category"] == category]
    #     lca.set_index("ProductSystemID", inplace=True)
    #     #lca.drop(columns=["ID","StandardDev","DataQuality"], inplace=True)



    for c in cats.keys():
        df_ps_impacts[c] = np.nan
        for idx in df_ps_impacts.index:
            df_ps_impacts.at[idx,c]=df_ps_impacts.at[idx,"Result"]*df_flows.at[idx,c]
    df_sum = df_ps_impacts.sum()

    # Recycling IDS
    if PS_ID-PS_ID % 1000 == 3000:
        ps_pyro = 910000+PS_ID
        ps_hydr = 930000+PS_ID
        ps_elec = 950000+PS_ID
        ps_biol = 970000+PS_ID
    else:
        ps_pyro = 920000+PS_ID
        ps_hydr = 940000+PS_ID
        ps_elec = 960000+PS_ID
        ps_biol = 980000+PS_ID
    rename = {ps_pyro: "Pyro",
            ps_hydr: "Hydr",
            ps_elec: "Elec",
            ps_biol: "BioL"}


    ps_list_recycling = [ps_pyro, ps_hydr, ps_elec, ps_biol]
    ps_list_recycling = [int(p) for p in ps_list_recycling]
    df_recovery = pd.DataFrame(index=ps_list_recycling)

    for cat in cats.keys():
        impacts = get_lca_contributions(
            ps_list=ps_list_recycling, category=cat, group=9000)
        df_recovery[cat] = np.nan
        for idx in df_recovery.index:
            df_recovery.at[idx, cat] = -impacts.at[idx, "Result"]

    scenarios = ["Potential", "Manual", "Deep", "Mech.", "Mix"]
    factors = [1, 0.8, 0.97, 0.26, 0.7]
    factors = [f/0.97 for f in factors]
    results = {}
    for cat in cats.keys():
        df_recovery_cat = pd.DataFrame(index=ps_list_recycling)
        for i in range(0, 5):
            df_recovery_cat[scenarios[i]] =  df_recovery[cat]*factors[i]/df_sum[cat]
        df_recovery_cat.index = ["Pyrom.","Hydrom.", "El-Rec.", "BioLeech"]# = df_recovery_cat.rename(index=rename, inplace=True)
        results[cats[cat]] = df_recovery_cat

    cats_plot = {"CED - total": "CED",
            "CExD - metals": "CExD",
            "EI-Minerals- Total": "ADP-EI",
            "GPR-Minerals- Total": "GPSR",
            "CML-climate change (GWP 100a)": "GWP",
            "ReCiPe Endpoint (E - A) - total (total)": "Total",
            }



    fig,axs= plt.subplots(3,2,figsize=(8,10))
    ax_ = [axs[i,j] for i in range(3) for j in range(2)]
    for i in range(0,6):
        ax = ax_[i]
        name = list(cats_plot.keys())[i]
        symbol = cats[name]
        df_plot= results[symbol].transpose()
        embodied_impact = 1# df_sum[name]
        df_plot.plot.bar(ax=ax,cmap="winter")
        ax.axhline(y = embodied_impact, color = 'grey', label = "Embodied Impact", linestyle="--")
        ax.legend(fontsize=8)
        #ax.set_title(name)
        label = full_label[cats[name]]
        ax.set_ylabel(label)
        ax.tick_params(axis='x', rotation=0)

        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(FONTSIZE)
        if i != 5:
            ax.get_legend().remove()
        else:
            ax.legend(bbox_to_anchor=(-0.1, -0.3),
                    loc="lower center", fontsize=FONTSIZE, ncols=5)
            
        #ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0E}'))

        for label in ax.get_yticklabels():
            label.set_fontsize(FONTSIZE*0.75)
            label.set_x(0.01)

    fig.savefig(f"RecyclingScenarios/scenarios_{PS_ID}.png", bbox_inches='tight', dpi=1200)
    plt.close()